##Design and implement a scalable Natural Language Processing (NLP) pipeline 
on the Databricks using SpaCy or NLTK for extracting insights from unstructured customer feedback data.
The goal is to preprocess, analyze, and extract meaningful patterns such as sentiment, named entities, 
and key themes that can be used to improve product strategy and customer satisfaction.
 
Please feel free to pick any open dataset/create one instead.

In [0]:
%pip install numpy==1.26.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/18.3 MB ? eta -:--:--
   ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/18.3 MB 10.3 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/18.3 MB 19.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


---------------------------------------------------------------------------
SystemError                               Traceback (most recent call last)
File /databricks/python_shell/lib/dbruntime/PipMagicOverrides.py:175, in PipMagicHandler.runCmd(self, magicCmd, line)
    172         self.entry_point.getLogger()._logInfo(
    173             f"{__name__}: Setting spark.databricks.connect.environment to {serialized_env}."
    174         )
--> 175         spark.conf.set("spark.databricks.connect.environment", serialized_env)
    176 except Exception as e:

File /databricks/python_shell/lib/dbruntime/UserNamespaceInitializer.py:43, in TripWire.__getattr__(self, item)
     42 def __getattr__(self, item):
---> 43     self._raise()

File /databricks/python_shell/lib/dbruntime/UserNamespaceInitializer.py:33, in TripWire._raise(self)
     32 if self.msg == 'not_implemented' and self.name == 'spark':
---> 33     raise SystemError(self._UNINITIALIZED_INTERNAL_ERROR.format(self.name))
     34 i

In [0]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
%restart_python

## 1. Import and Load NLP Models

In [0]:
import spacy
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType

nlp = spacy.load("en_core_web_sm")


## 2. Create Sample Feedback Data

In [0]:
data = [
    ("The app is really slow and crashes often. Support team didn’t help much.",),
    ("I love the new update! The interface is sleek and very user-friendly.",),
    ("Too expensive for the features offered. Considering switching to CompetitorX.",),
    ("Great customer service and fast delivery. Keep up the good work!",),
    ("Terrible experience. I want a refund. Product quality was poor.",),
    ("The chatbot was surprisingly helpful. Solved my issue in 5 minutes!",),
    ("Shipping took way too long. Not satisfied.",),
    ("Amazing features and value for money. Totally worth it!",),
    ("I had issues with payment but they resolved it quickly.",),
    ("Never received the product. Very disappointed.",)
]

schema = ["feedback_text"]
df = spark.createDataFrame(data, schema)


In [0]:
df.display()

feedback_text
The app is really slow and crashes often. Support team didn’t help much.
I love the new update! The interface is sleek and very user-friendly.
Too expensive for the features offered. Considering switching to CompetitorX.
Great customer service and fast delivery. Keep up the good work!
Terrible experience. I want a refund. Product quality was poor.
The chatbot was surprisingly helpful. Solved my issue in 5 minutes!
Shipping took way too long. Not satisfied.
Amazing features and value for money. Totally worth it!
I had issues with payment but they resolved it quickly.
Never received the product. Very disappointed.


## 3. Define UDFs for NLP

#### Named Entity Recognition (NER)

In [0]:
def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

entity_udf = udf(extract_entities, ArrayType(StringType()))


#### Sentiment Analysis (TextBlob)

In [0]:
def extract_sentiment(text):
    return TextBlob(text).sentiment.polarity

sentiment_udf = udf(extract_sentiment, StringType())


#### Theme Extraction (via Noun Chunks)

In [0]:
def extract_noun_chunks(text):
    doc = nlp(text)
    return [chunk.text for chunk in doc.noun_chunks]

noun_chunk_udf = udf(extract_noun_chunks, ArrayType(StringType()))


## 4. Apply NLP Pipeline

In [0]:
result_df = df.withColumn("entities", entity_udf("feedback_text")) \
              .withColumn("sentiment_score", sentiment_udf("feedback_text")) \
              .withColumn("themes", noun_chunk_udf("feedback_text"))


In [0]:
result_df.display()


feedback_text,entities,sentiment_score,themes
The app is really slow and crashes often. Support team didn’t help much.,List(),-0.05000000000000002,"List(The app, Support team)"
I love the new update! The interface is sleek and very user-friendly.,List(),0.2901515151515151,"List(I, the new update, The interface)"
Too expensive for the features offered. Considering switching to CompetitorX.,List(),-0.5,List(the features)
Great customer service and fast delivery. Keep up the good work!,List(),0.625,"List(Great customer service, fast delivery, the good work)"
Terrible experience. I want a refund. Product quality was poor.,List(),-0.7,"List(Terrible experience, I, a refund, Product quality)"
The chatbot was surprisingly helpful. Solved my issue in 5 minutes!,List(5 minutes),0.875,"List(The chatbot, my issue, 5 minutes)"
Shipping took way too long. Not satisfied.,List(),-0.15,List(Shipping)
Amazing features and value for money. Totally worth it!,List(),0.48750000000000004,"List(Amazing features, value, money)"
I had issues with payment but they resolved it quickly.,List(),0.3333333333333333,"List(I, issues, payment, they, it)"
Never received the product. Very disappointed.,List(),-0.9750000000000001,List(the product)
